In [1]:
import matplotlib.pylab as plt
# import networkx as nx
import numpy as np
import pandas as pd
import time
import sys
import os
import datetime

# preamble for jupyter notebook and django
import django
import platform
from django.db.models import Q, F, Sum, Count, FloatField, Case, When

if platform.node() == "mcc-apsis":
    #sys.path.append('/home/muef/tmv/BasicBrowser/')
    sys.path.append('/var/www/tmv/BasicBrowser/')
else:
    # local paths
    sys.path.append('/media/Data/MCC/tmv/BasicBrowser/')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
from django.core import management
import twitter.models as twm

In [43]:
# filepath = "../Parliament Australia/every_politician_database-popolo-v1.0.json"

# source: http://everypolitician.org/uk/commons/download.html
filepath = "everypolitician_UK-popolo-v1.0.json"

country = "United Kingdom"
parliament_name = "House of Commons"
parlperiod = 57

#pd.read_json(filepath)
import json

In [45]:
with open("countries.json") as json_data:
    cdata = json.load(json_data)

# print([d['country'] for d in cdata])

for d in cdata:
    if (d['country'] == country or d['name'] == country):
        print(d)
        for leg in d['legislatures']:
            if leg['name'] == parliament_name or leg['slug'] == parliament_name:
                json_url = leg['popolo_url']
                print(json_url)
                for pp in leg['legislative_periods']:
                    if pp['slug'] == str(parlperiod):
                        print(pp)

{'name': 'United Kingdom', 'country': 'United Kingdom', 'code': 'GB', 'slug': 'UK', 'legislatures': [{'name': 'House of Commons', 'slug': 'Commons', 'sources_directory': 'data/UK/Commons/sources', 'popolo': 'data/UK/Commons/ep-popolo-v1.0.json', 'popolo_url': 'https://cdn.rawgit.com/everypolitician/everypolitician-data/ee0feea9035346a1f9091f2211527107317f007f/data/UK/Commons/ep-popolo-v1.0.json', 'names': 'data/UK/Commons/names.csv', 'lastmod': '1543329079', 'person_count': 1436, 'sha': 'ee0feea9035346a1f9091f2211527107317f007f', 'legislative_periods': [{'id': 'term/57', 'name': '57th Parliament of the United Kingdom', 'start_date': '2017-06-09', 'slug': '57', 'csv': 'data/UK/Commons/term-57.csv', 'csv_url': 'https://cdn.rawgit.com/everypolitician/everypolitician-data/ee0feea9035346a1f9091f2211527107317f007f/data/UK/Commons/term-57.csv'}, {'end_date': '2017-05-03', 'id': 'term/56', 'name': '56th Parliament of the United Kingdom', 'start_date': '2015-05-08', 'slug': '56', 'csv': 'data/U

In [19]:
def build_handle_list(filepath, org_id = '8bbf6031-aa59-4b0a-80b2-1cdc81e4047d', parlperiod = 57, verbosity=0):
    # Reading the json as a dict
    with open(filepath) as json_data:
        data = json.load(json_data)
    members_of_parliament = [d['person_id'] for d in data['memberships']
                         if d['organization_id'] == org_id and
                            d['legislative_period_id'] == 'term/' + str(parlperiod)]
    print("number of parliamentarians: {}".format(len(members_of_parliament)))
    
    df = pd.DataFrame(columns=["surname", "first_name", "name", "birth_date", "twitter_url", "handle",
                           "wikipedia", "in_current_parliament"])

    for i, person in enumerate(data['persons']):
        #print(person)
        twitter = None
        twitter_handle = None
        wikipedia = None
        if person['id'] in members_of_parliament:
            in_parliament = 1
        else:
            in_parliament = 0

        for link in person['links']:
            if link['note'] == 'twitter':
                twitter = link['url']
                twitter_handle = "@" + twitter.split('/')[3]
            elif link['note'] == 'wikipedia':
                wikipedia = link['url']
        try:
            fname = person['family_name']
        except:
            if verbosity > 0:
                print('warning: key not found (family_name)')
            fname = None
        try:
            bdate = person['birth_date']
        except:
            if verbosity > 0:
                print('warning: key not found (birth_date)')
            bdate = None
        try:
            gname = person['given_name']
        except:
            gname = None
            if verbosity > 0:
                print('warning: key not found (given_name)')

        df.loc[i] = [fname, gname, person['name'],
                     bdate, twitter, twitter_handle, wikipedia, in_parliament]

    counter = 0
    cp_df = pd.DataFrame(columns=["surname", "first_name", "name", "birth_date", "twitter_url", "handle",
                               "wikipedia", "in_current_parliament"])

    for i, row in df.iterrows():
        if row['handle'] and row['in_current_parliament']:
            if verbosity > 0:
                print(row['handle'])
            cp_df.loc[i] = row
            counter += 1
    print("parliamentarians with twitter account: {}".format(counter))
    cp_df = cp_df.reset_index()
    return cp_df

In [20]:
table = build_handle_list(filepath)
table.to_csv("Twitter_list_UK_MPs_everypolitician.csv")

number of parliamentarians: 654
parliamentarians with twitter account: 588


In [15]:
table

,index,surname,first_name,name,birth_date,twitter_url,handle,wikipedia,in_current_parliament
0,0,Frith,James,James Frith,1977-04-23,https://twitter.com/JamesFrith,@JamesFrith,None,1
1,1,None,Anna,Anna McMorrin,1971,https://twitter.com/annamcmorrin,@annamcmorrin,None,1
2,6,Dinenage,Caroline,Caroline Dinenage,1971-10-28,https://twitter.com/cj_dinenage,@cj_dinenage,https://en.wikipedia.org/wiki/Caroline_Dinenage,1
3,7,Cowan,Ronnie,Ronnie Cowan,1959-09-06,https://twitter.com/ronniecowan,@ronniecowan,https://en.wikipedia.org/wiki/Ronnie_Cowan_%28...,1
4,10,None,Daniel,Daniel Zeichner,1956-11-09,https://twitter.com/danielzeichner,@danielzeichner,https://en.wikipedia.org/wiki/Daniel_Zeichner,1
5,12,None,Thomas,Tom Brake,1962-05-06,https://twitter.com/thomasbrake,@thomasbrake,https://en.wikipedia.org/wiki/Tom_Brake,1
6,14,None,Pat,Pat McFadden,1965-03-26,https://twitter.com/patmcfaddenmp,@patmcfaddenmp,https://en.wikipedia.org/wiki/Pat_McFadden_(Br...,1
7,17,Smith,Laura,Laura Smith,None,https://twitter.com/laurasmithmp,@laurasmithmp,None,1
8,18,None,Alec,Alec Shelbrooke,1976-01-10,https://twitter.com/AlecShelbrooke,@AlecShelbrooke,https://en.wikipedia.org/wiki/Alec_Shelbrooke,1
9,23,Day,Martyn,Martyn Day,1971-03-26,https://twitter.com/MartynDaySNP,@MartynDaySNP,https://en.wikipedia.org/wiki/Martyn_Day_%28po...,1


In [21]:
for handle in table['handle'].dropna()[:-1]:
    print(handle.strip())
    management.call_command('add_user', handle.strip())

@JamesFrith


TweepError: [{'code': 89, 'message': 'Invalid or expired token.'}]

In [13]:
# check if monitoring is set appropriately

handles = table['handle'].dropna()[:-1].values
screen_names = [h.strip('@') for h in handles]

print("Users: {}".format(len(handles)))
twm.User.objects.filter(screen_name__in=screen_names).values('screen_name', 'monitoring')

Users: 133


<QuerySet [{'screen_name': 'AlanTudgeMP', 'monitoring': True}, {'screen_name': 'AlboMP', 'monitoring': True}, {'screen_name': 'AmandaRishworth', 'monitoring': True}, {'screen_name': 'AndrewGeeMP', 'monitoring': True}, {'screen_name': 'andrewjgiles', 'monitoring': True}, {'screen_name': 'AndrewLamingMP', 'monitoring': True}, {'screen_name': 'Ann_SudmalisMP', 'monitoring': True}, {'screen_name': 'AnthonyByrne_MP', 'monitoring': True}, {'screen_name': 'Barnaby_Joyce', 'monitoring': True}, {'screen_name': 'billshortenmp', 'monitoring': True}, {'screen_name': 'BOConnorMP', 'monitoring': True}, {'screen_name': 'Bowenchris', 'monitoring': True}, {'screen_name': 'broad4mallee', 'monitoring': True}, {'screen_name': 'BroadbentMP', 'monitoring': True}, {'screen_name': 'CatherineKingMP', 'monitoring': True}, {'screen_name': 'ChrisCrewtherMP', 'monitoring': True}, {'screen_name': 'ClareONeilMP', 'monitoring': True}, {'screen_name': 'c_otoole', 'monitoring': True}, {'screen_name': 'cporterwa', 'moni